In [1]:
import datetime
import QuantExt as qe # 퀀트립 확장판 불러오기
from module.FX_Curve import GET_QUOTE, USDIRS_CURVE, KRWCCS_CURVE # 이전에 만든 함수들 가져오기(여기선 Fx_curve)

In [2]:
class FXF():
    def __init__(self, todays_date, maturity_date, fx_spot, fx_forward, usd_notional, position): # 초기화 진행
        
        # Initial Setup 1 - Date / Curves / FX Spot
        self.date = todays_date # 오늘 잘짜
        self.usd_curve = self.USD_CURVE(self.date) # 달러 IRS 커브
        self.krw_curve = self.KRW_CURVE(self.date) # 원화 CCS 커브
        self.fx_spot = fx_spot
        
        # Initial Setup 2 - Instrument Info
        self.maturity_date = qe.Date(maturity_date.day, maturity_date.month, maturity_date.year) # 퀀트립 형태의 날짜로 바꿈
        self.usd = qe.KRWCurrency() # 선도 환율
        self.usd_notional = usd_notional # 달러 명목 금액
        self.krw = qe.USDCurrency()
        self.krw_notional = usd_notional * fx_forward
        self.day_count = qe.ActualActual() # 날짜 카운트
        
        if position == 'Long': # 포지션 방향(매수 or 매도)
            self.payCcy1 = True # payCcy1 ==> 포지션의 방향을 지정하기 위해 존재(True는 원화를 내고 달러를 받는 선도환 매수를 의미)
        else:
            self.payCcy1 = False # 여기서 False는 원화를 받고 달러를 내는 선도환 매도를 의미
        
        # Pricing Results
        self.npv = self.PRICING(self.usd_curve, self.krw_curve, self.fx_spot) # 선도환 이론가 계산 함수
        self.fx_delta = self.FX_DELTA() # 환율이 1% 움직였을 때 선도환의 환델타 계산
        self.usd_ir_delta = self.USD_IR_DELTA() # 달러 금리가 1bp 움직였을 때 선도환의 달러 금리 델타 계산
        self.krw_ir_delta = self.KRW_IR_DELTA() # 원화 금리가 1bp 움직였을 때 선도환의 달러 금리 델타 계산
        self.theta = self.THETA() # 날짜가 하루 지났을 때 선도환의 세타 계산
        
    def USD_CURVE(self, date): # Fx_curve에서 구현한 USD_CURVE(IRS) 불러오는 함수
        return USDIRS_CURVE(date, GET_QUOTE(date, 'USD'))
    
    def KRW_CURVE(self, date): # Fx_curve에서 구현한 KRW_CURVE(CCS) 불러오는 함수
        return KRWCCS_CURVE(date, GET_QUOTE(date, 'KRW'))
        
    def PRICING(self, usd_curve, krw_curve, fx_spot): # 실제 선도환의 이론가를 계산하는 기능을 담당하는 함수
        usd_curve_handle = qe.YieldTermStructureHandle(usd_curve)
        krw_curve_handle = qe.YieldTermStructureHandle(krw_curve)
        fx_spot_handle = qe.QuoteHandle(qe.SimpleQuote(fx_spot))
        
        fxf = qe.FxForward(self.krw_notional,
                           self.krw,
                           self.usd_notional,
                           self.usd,
                           self.maturity_date,
                           self.payCcy1) # FxForward() ==> 퀀트립 확장판에 존재하는 선도환 구현 매서드

        # To-do : Dual Curve Import
        engine = qe.DiscountingFxForwardEngine(self.krw,
                                               krw_curve_handle,
                                               self.usd,
                                               usd_curve_handle,
                                               fx_spot_handle) # DiscountingFxForwardEngine() ==> 선도환을 평가할 수 있는 매서드
        
        fxf.setPricingEngine(engine)
        npv = fxf.NPV()
        return npv

    def FX_DELTA(self):
        # 1%
        percentage = 0.01 # 환율 1%를 의미
        
        up_fx = self.fx_spot * (1 + percentage)
        up_fxf = self.PRICING(self.usd_curve, self.krw_curve, up_fx)
        
        down_fx = self.fx_spot * (1 - percentage)
        down_fxf = self.PRICING(self.usd_curve, self.krw_curve, down_fx)
        
        fx_delta = (up_fxf - down_fxf) / 2
        return fx_delta
    
    def USD_IR_DELTA(self): # 선도환의 달러 금리 델타
        curve_handle = qe.YieldTermStructureHandle(self.usd_curve)
        
        # 1bp
        basis_point = 0.0001 # 0.01%를 의미
        
        # FRA price when 1bp up
        up_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        up_fxf = self.PRICING(up_curve, self.krw_curve, self.fx_spot)
        
        # FRA price when 1bp down
        down_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(-basis_point)))
        down_fxf = self.PRICING(down_curve, self.krw_curve, self.fx_spot)

        # USD Curve Delta
        delta = (up_fxf - down_fxf) / 2
        return delta
    
    def KRW_IR_DELTA(self): # 선도환의 원화 금리 델타
        curve_handle = qe.YieldTermStructureHandle(self.krw_curve)
        
        # 1bp
        basis_point = 0.0001 # 0.01%를 의미
        
        # FRA price when 1bp up
        up_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        up_fxf = self.PRICING(self.usd_curve, up_curve, self.fx_spot)
        
        # FRA price when 1bp down
        down_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(-basis_point)))
        down_fxf = self.PRICING(self.usd_curve, down_curve, self.fx_spot)

        # KRW Curve Delta
        delta = (up_fxf - down_fxf) / 2
        return delta
    
    def THETA(self):
        price_t0 = self.PRICING(self.usd_curve, self.krw_curve, self.fx_spot)
        usd_curve_t1 = self.USD_CURVE(self.date + datetime.timedelta(days=1))
        krw_curve_t1 = self.KRW_CURVE(self.date + datetime.timedelta(days=1))
        price_t1 = self.PRICING(usd_curve_t1, krw_curve_t1, self.fx_spot)
        
        theta = price_t1 - price_t0
        return theta

In [3]:
if __name__ == "__main__":
    
    # Today's Date
    todays_date = datetime.date(2020, 10, 9)
    
    # FX Spot Rate
    fx_spot = 1153.30
    
    # FX Forward Instrument
    fx_forward = 1152.32
    maturity_date = datetime.date(2021, 10, 9)
    usd_notional = 10000000
    krw_notional = usd_notional * fx_forward
    position = 'Long'
    
    # Build FXF Object
    fxf = FXF(todays_date,
              maturity_date,
              fx_spot,
              fx_forward,
              usd_notional,
              position)

    # Print the Results
    print("Price = {}".format(round(fxf.npv, 4)))
    print("FX Delta = {}".format(round(fxf.fx_delta, 4)))
    print("USD IR Delta = {}".format(round(fxf.usd_ir_delta, 4)))
    print("KRW IR Delta = {}".format(round(fxf.krw_ir_delta, 4)))
    print("Theta = {}".format(round(fxf.theta, 4)))

Price = -98775.3408
FX Delta = 115055468.771
USD IR Delta = -1166534.6159
KRW IR Delta = 1150564.5672
Theta = -47815.8777
